In [ ]:
#Importing Libraries
import tensorflow
import numpy as np
import pandas as pd
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
import os
from music21 import converter
import kagglehub
np.random.seed(42)
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.models import load_model
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




```
content_dir = "classical-music-midi"

if not os.path.exists(content_dir):
    os.makedirs(content_dir)  # Create the content directory if it doesn't exist

shutil.copytree(path2, content_dir, dirs_exist_ok=True)
print(f"Dataset files moved to: {content_dir}")

```





```
filepath = os.path.join(content_dir, "albeniz")

# Initialize a list to store all MIDI streams
all_midis = []

# Iterate over all files in the Chopin folder and parse MIDI files

for filename in os.listdir(filepath):
    if filename.endswith(".mid"):
        midi_path = os.path.join(filepath, filename)
        midi = converter.parse(midi_path)
        all_midis.append(midi)

print(f"Loaded {len(all_midis)} MIDI files.")"""
```





```
Corpus = extract_notes(midis)

# Function to extract notes and chords from MIDI files

def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes


print("Extracting notes. This may take some time...")
```





```


# Save the notes as a pickle file
notes_pkl_path = os.path.join("# PATH ", f"{composer_name}_notes.pkl")
with open(notes_pkl_path, 'wb') as f:
    pickle.dump(Corpus, f)

print(f"Extracted notes saved as {notes_pkl_path}")"""

```



In [ ]:
composer_name = "chopin"

We have our data in the form of a corpus. A list of strings, if you will. Each string indicates a musical note. Let us explore this data corpus.

**In this section**
* Exploring the data Corpus
* Examine all the notes in the Corpus
* Simplifying our Corpus to Built a working model

**Have a look at the first 50 values in our corpus**

In [ ]:
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes

with open(os.path.join("/content/drive/MyDrive/MIDI/", f"{composer_name}_MIDI.pkl"), 'rb') as f:

  midi = pickle.load(f)

Corpus =   extract_notes(midi)

print("First fifty values in the Corpus:", Corpus[:50])


First fifty values in the Corpus: ['F3', 'F2', 'G3', 'G2', 'B-3', 'B-2', 'C#4', 'C#3', 'E3', 'E2', 'G3', 'G2', 'F3', 'F2', 'G3', 'G2', 'B-3', 'B-2', 'C#4', 'C#3', 'E3', 'E2', 'G3', 'G2', 'G3', 'G2', 'A3', 'A2', 'C4', 'C3', 'E-4', 'E-3', 'F#3', 'F#2', 'A3', 'A2', 'G3', 'G2', 'A3', 'A2', 'C4', 'C3', 'E-4', 'E-3', 'F#3', 'F#2', 'A3', 'A2', 'B-3', 'B-2']


In [ ]:
#Creating a count dictionary
count_num = Counter(Corpus)
print("Total unique notes in the Corpus:", len(count_num))

Total unique notes in the Corpus: 397


**Examine all the notes in the Corpus**

In [ ]:
#Getting a list of rare chords
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)
print("Total number of notes that occur less than 100 times:", len(rare_note))
for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)

print("Length of Corpus after elemination the rare notes:", len(Corpus))

Total number of notes that occur less than 100 times: 270
Length of Corpus after elemination the rare notes: 53745


In [ ]:
# Initialize an empty dictionary to store unique characters
unique_chars = {}

# Iterate through the corpus to collect unique characters
for char in Corpus:
    if char not in unique_chars:
        unique_chars[char] = None

# Convert the unique characters to a sorted list
symb = sorted(unique_chars.keys())

# Lengths of the corpus and unique characters
L_corpus = len(Corpus)  # Total number of characters in the corpus
L_symb = len(symb)      # Number of unique characters

# Build mapping dictionaries
mapping = {char: idx for idx, char in enumerate(symb)}
reverse_mapping = {idx: char for idx, char in enumerate(symb)}

# Print the results
print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)


Total number of characters: 53745
Number of unique characters: 255


**Encoding and Splitting the Corpus as Labels and Targets**

In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])


L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 53705


In [ ]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb)
# one hot encode the output variable
y = tensorflow.keras.utils.to_categorical(targets)



```
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

#Model's Summary
model.summary()
```





```
X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, batch_size=64, epochs=200)
model.save("model_path")
```



In [ ]:
#First Lets write some functions that we need to look into the data

def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = []
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)
    return Melody_midi

Melody_Snippet = chords_n_notes(Corpus[:100])


**Generating the Melody**

A function to obtain the generated music

In [ ]:
def Malody_Generator(Note_Count,composer_name):
    mdl_dir = '/content/drive/MyDrive/MIDI/Model'  # Update this path if needed
    mdl_filename = f"{composer_name}.keras"
    model_path = os.path.join(mdl_dir, mdl_filename)
    model = load_model(model_path)
    X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,length,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.5 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(L_symb)
        Notes_Generated.append(index)
        Music = [
            reverse_mapping[char] if char in reverse_mapping else random.choice(list(reverse_mapping.values()))
            for char in Notes_Generated
        ]

        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100,composer_name)


In [ ]:
Melody.write('midi','Melody_Generated.mid')

'Melody_Generated.mid'